<a href="https://colab.research.google.com/github/Lee-geun-bum/polars/blob/main/polars_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# gpt 문제

In [14]:
import os

print(os.getcwd())


/content


In [16]:
df = pl.read_csv("tmp.csv")

df

user,task,trial,x,y,acc,valid
str,str,i64,i64,i64,i64,i64
"""U01""","""PS""",1,10,5,1,1
"""U01""","""PS""",2,8,4,1,1
"""U01""","""AS""",1,6,3,1,1
"""U01""","""AS""",2,6,2,0,1
"""U02""","""PS""",1,9,3,1,1
…,…,…,…,…,…,…
"""U03""","""AS""",2,8,4,1,1
"""U04""","""PS""",1,10,2,1,0
"""U04""","""PS""",2,5,5,1,1


In [33]:
tmp = (
    df
    .filter(pl.col("valid")==1)
    .group_by(["user", "task"])
    .agg(
        pl.col("x").sum().alias("sum_x"),
        pl.col("y").sum().alias("sum_y"),
        pl.col("acc").mean().alias("acc_rate")
    )
    .with_columns(
        (pl.col("sum_x") /pl.col("sum_y")).alias("ratio_xy")
        )
    .filter(pl.col("acc_rate") >= 0.7)
    .group_by("task")
    .agg(
        pl.col("user").n_unique().alias("n_users"),
        pl.col("ratio_xy").mean().alias("mean_ratio"),
        pl.col("sum_x").sum().alias("total_sum_x")
    )
)

tmp

task,n_users,mean_ratio,total_sum_x
str,u32,f64,i64
"""AS""",2,1.5,21
"""PS""",3,1.666667,41


ans

In [39]:

res = (
    df
    .filter(pl.col("valid") == 1)
    .group_by(["user", "task"])
    .agg(
        sum_x    = pl.col("x").sum(),
        sum_y    = pl.col("y").sum(),
        acc_rate = pl.col("acc").mean()
    )
    .with_columns(
        ratio_xy = pl.col("sum_x") / pl.col("sum_y")
    )
    .filter(pl.col("acc_rate") >= 0.7)
    .group_by("task")
    .agg(
        n_users     = pl.col("user").n_unique(),
        mean_ratio  = pl.col("ratio_xy").mean(),
        total_sum_x = pl.col("sum_x").sum()
    )
    .with_columns(pl.col("mean_ratio").round(4))
    .sort("task")
)

res


task,n_users,mean_ratio,total_sum_x
str,u32,f64,i64
"""AS""",2,1.5,21
"""PS""",3,1.6667,41


# 문제 2

In [38]:
df = pl.read_csv("tmp.csv")
user = pl.read_csv("user.csv")

df


user,task,trial,x,y,acc,valid
str,str,i64,i64,i64,i64,i64
"""U01""","""PS""",1,10,5,1,1
"""U01""","""PS""",2,8,4,1,1
"""U01""","""AS""",1,6,3,1,1
"""U01""","""AS""",2,6,2,0,1
"""U02""","""PS""",1,9,3,1,1
…,…,…,…,…,…,…
"""U03""","""AS""",2,8,4,1,1
"""U04""","""PS""",1,10,2,1,0
"""U04""","""PS""",2,5,5,1,1


In [62]:

res = (
    df
    .filter(pl.col("valid") == 1)
    .group_by(["user", "task"])
    .agg(
        sum_x    = pl.col("x").sum(),
        sum_y    = pl.col("y").sum(),
        acc_rate = pl.col("acc").mean()
    )
    .with_columns(
        ratio_xy = pl.col("sum_x") / pl.col("sum_y")
    )
    .filter(pl.col("acc_rate") >= 0.7)
    .join(user, on="user", how = "left")
    .group_by(["age", "task"])
    .agg(
        n_users     = pl.col("user").n_unique(),
        mean_ratio  = pl.col("ratio_xy").mean(),
        total_sum_x = pl.col("sum_x").sum()
    )
    .with_columns(pl.col("mean_ratio").round(4))
    .sort("task")
    .to_pandas()
)

res

,age,task,n_users,mean_ratio,total_sum_x
0,MCI,AS,1,2.0,16
1,CN,AS,1,1.0,5
2,CN,PS,2,2.0,36
3,MCI,PS,1,1.0,5


In [61]:
## .to_pandas()만 하면 어짜피 나중에 뭐 호환성 문제는 고려안될듯

res_pd = res.to_pandas()
res_pd

,user,task,sum_x,sum_y,acc_rate,ratio_xy,age
0,U04,PS,5,5,1.0,1.0,MCI
1,U02,PS,18,9,1.0,2.0,CN
2,U02,AS,5,5,1.0,1.0,CN
3,U01,PS,18,9,1.0,2.0,CN
4,U03,AS,16,8,1.0,2.0,MCI
